# 인공신경망의 Back-propagation 을 해봅시다

이번 과제의 목표는 Back-propagation 을 계산하고 이를 torch의 자동 미분과 비교하여 맞는지 확인하는 것입니다. 

## 1) 필수 라이브러리와 함수를 불러 옵니다.

In [87]:
import torch
import numpy as np

In [88]:
def ReLU_func(outputs):
    zero_tensor = torch.zeros(outputs.size())
    final_outputs = torch.maximum(outputs,zero_tensor)

    return final_outputs

def softmax(outputs):
    numerator = torch.exp(outputs - torch.max(outputs,axis=1)[0].view(-1,1))
    denominator = torch.sum(numerator, axis=1).view(-1,1)
    softmax = numerator/denominator
    
    return softmax

def cross_entropy(outputs, labels):
    return torch.sum(-1*labels*torch.log(outputs),axis=1)

## 2) 인공신경망 계산을 합니다. 

입력은 [4,1], Label은 [0,1], W_0 은 $\pmatrix{1,-2 \\ 2,5}$, W_1은  $\pmatrix{3,-3 \\ -1,1}$ 로 주어졌을 때

각 단계를 h, L, O, s, l 를 각각 역치 전 값, 히든 레이어 값, 출력값, 소프트맥스 후, loss로 하여 계산합니다.
각 값은 torch.tensor 클래스의 객체로 만들어 자동미분이 가능하게 만드세요.

In [89]:
I = torch.tensor([4.,1.], requires_grad = True)
label = torch.tensor([0.,1.], requires_grad = True)
W_0 = torch.tensor([[1.,-2.],[2.,5.]], requires_grad = True)
W_1 = torch.tensor([[3.,-3.],[-1.,1.]], requires_grad = True)

In [90]:
h =  torch.matmul(W_0,I)
L =  ReLU_func(h)
O =  torch.matmul(W_1, L)
s =  softmax(O.reshape(1,-1))
l =  cross_entropy(s.unsqueeze(0), label.unsqueeze(0))

In [91]:
print(l)

tensor([[0., 0.]], grad_fn=<SumBackward1>)


In [92]:
l.mean().backward()

## 3) $\nabla_{W_0}l$ 계산하기

위에서 주어진 h, L, O, s, l을 이용하여 $\nabla_{W_0}l$를 계산해봅시다. 

numpy를 이용하여 계산하세요!

참고로 

 $$\nabla_s l = \pmatrix{-\frac{label_0}{s_0}, -\frac{label_1}{s_1} } $$
 $$ \nabla_o s = \pmatrix{s_0(1-s_0), -s_0s_1 \\ -s_1s_0, s_1(1-s_1) }$$
 $$ \nabla_{w_1} o = \pmatrix{L_0,0,L_1,0 \\ 0, L_0, 0, L_1 }$$

In [93]:
s_l= np.array([-label[0].item() / s[0].item(), -label[1].item() / s[1].item()])
o_s= np.array([s[0]*(1-s[0]), -s[0]*s[1]-s[1]*s[0], s[1]*(1-s[1])])
w1_o= np.array([[L[0].item(), 0], [0, L[1].item()]])

ValueError: only one element tensors can be converted to Python scalars

In [94]:
w1_l =  W_1.t().matmul(torch.tensor([o_s]).t() * torch.tensor([s_l])).t()# 채워넣으시오
print(w1_l)

NameError: name 'o_s' is not defined

자동 미분 결과와 비교해 봅시다.

In [100]:
W_1.grad # 채워넣으시오

tensor([[ 7.7811e-20,  5.0577e-19],
        [-7.7811e-20, -5.0577e-19]])

자동 미분 결과와 비교해보고 차이점이 무엇인지 그리고 왜 그런 결과가 생겼는지 서술하시오.

정답) numpy로 계산한 결과는 벡터와 자동미분한 결과는 매트릭스로 결과가 나왔다. numpy로 계산한 결과는 w1_o의 정의에 따라 벡터로 출력되며, 자동 미분은 입력의 차원으로 계산되기 때문이다.

## 4) $\nabla_{w_0} l $ 계산하기

위에서 주어진 h, L, O, s, l을 이용하여 $\nabla_{W_0}l$를 계산해봅시다. 

numpy를 이용하여 계산하세요!

참고로 

 $$\nabla_L o = W_1^T $$
 $$ \nabla_h L = \pmatrix{1 or 0, 0 \\ 0, 1 or 0 }$$
 $$ \nabla_{w_0} h = \pmatrix{I_0,0,I_1,0 \\ 0, I_0, 0, I_1 }$$

In [96]:
L_o =  W_1.t()# 채워넣으시오
h_L = np.array([[1, 0], [0, 1]]) if h[0] > 0 else np.array([[0], [0]]) # 채워넣으시오) 
w0_h = np.array([I[0], 0, I[1], 0]).reshape(2,2).T # 채워넣으시오) 

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [97]:
w0_l =  np.array([L[0], 0, 0, L[1]]).reshape(2,2).T# 채워넣으시오
print(w0_l)

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

자동미분과 비교해봅시다.

In [101]:
W_0.grad # 채워넣으시오

tensor([[ 6.2249e-19,  1.5562e-19],
        [-6.2249e-19, -1.5562e-19]])

In [99]:
print("이서영 2020125046")

이서영 2020125046
